<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import torch, torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

import pandas as pd
from IPython.display import display
from pathlib import Path


# убрала все предупреждения, потому что не смоогла вырубить предупреждения для модификации summary для коротких промптов
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

"""
Скрипт сравнения распределений P(next_token) с поддержкой:
----------------------------------------------------------------
1. **Внутренних промптов** (список в коде) **или**
2. **Промптов из текстового файла** в той же директории.

Флаг `USE_FILE_PROMPTS` выбирает источник.
Если `True`, файл `PROMPT_FILE` читается целиком и используется как **один** длинный prompt.

Модификация «summary + слова» печатает краткое summary исходного prompt‑а.
Добавлена **безопасная токенизация**: если длиннее лимита модели (1024 для GPT‑2),
текст автоматически усечётся, а в консоль выведется предупреждение.
"""

# ---------- ПАРАМЕТРЫ ----------
MODEL_NAME   = "gpt2"
STEPS        = 3
DEVICE       = "cuda" if torch.cuda.is_available() else "cpu"

# Флаг, управляющий включением суммаризации как отдельной модификации.
SUMMARY_ENABLED = True

# Суммаризация
SUMM_MODEL   = "sshleifer/distilbart-cnn-12-6"
CUSTOM_WORDS = "Сводка:"

# Выбор источника промптов
USE_FILE_PROMPTS = False
PROMPT_FILE      = "prompt.txt"

# ---------- 1. Модели ----------
_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
_model     = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(DEVICE).eval()

_summarizer = pipeline(
    "summarization",
    model=SUMM_MODEL,
    tokenizer=SUMM_MODEL,
    device=0 if DEVICE=="cuda" else -1,
)

CTX_LIMIT = _model.config.n_positions

# ---------- 2. Вспомогательные функции ----------
def _next_token_probs(ids):
    with torch.no_grad():
        logits = _model(ids).logits[0, -1]
    return torch.softmax(logits, dim=-1).cpu()

def _kl(p, q, eps=0):  # нормально работает при eps=1e-8
    p = p + eps
    q = q + eps
    return torch.sum(p * torch.log(p / q)).item()

def _cos(p, q):
    return F.cosine_similarity(p, q, dim=0).item()

def _multi_step(prompt: str, n: int):
    token_ids = _tokenizer.encode(prompt)
    if len(token_ids) > CTX_LIMIT:
        print(f"⚠️ Промпт содержит {len(token_ids)} токенов и будет усечён до {CTX_LIMIT}.")
    ids = _tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=CTX_LIMIT
    )["input_ids"].to(DEVICE)

    dists = []
    for _ in range(n):
        probs = _next_token_probs(ids)
        dists.append(probs)
        next_id = probs.argmax().unsqueeze(0).unsqueeze(0).to(DEVICE)
        ids = torch.cat([ids, next_id], dim=1)
    return dists

# ---------- 3. Источник промптов и вывод top‑5 токенов ----------
PROMPTS = []
if USE_FILE_PROMPTS:
    prompt = Path(PROMPT_FILE).read_text(encoding="utf-8").strip()
    PROMPTS = [prompt]
else:
    PROMPTS = [
        "What are the health benefits of green tea?",
        "Explain the process of photosynthesis.",
        "How can I improve my time management skills?"
    ]

MODS = [
    ("original", lambda p: p),
    ("typo first e", lambda p: p.replace("e", "3", 1)),
    ("add salutation", lambda p: "Dear user, " + p),
    ("префикс ======", lambda p: "="*10 + p),
    ("префикс вопрос", lambda p: "I have a question. " + p),
    ("суффикс 10 лет?", lambda p: p + " in the next decade?"),
    ("summary", lambda p: CUSTOM_WORDS + _summarizer(p, max_length=60, min_length=10, do_sample=False)[0]['summary_text'])
]



C:\Users\cab\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\cab\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
PROMPTS

['What are the health benefits of green tea?',
 'Explain the process of photosynthesis.',
 'How can I improve my time management skills?']

In [3]:
#попробуем этот кусок переделать
# ----------  Выводим summary для каждого промпта  ----------
for i, base_prompt in enumerate(PROMPTS, 1):
    # ищем именно модификацию 'summary'
    for mod_name, mod_fn in MODS:
        if mod_name != "summary":
            continue                # пропускаем остальные модификации

        # 1. получаем строку с префиксом
        summary_with_prefix = mod_fn(base_prompt)

        # 2. убираем CUSTOM_WORDS
        summary_only = summary_with_prefix[len(CUSTOM_WORDS):]

        # 3. печатаем
        short_prompt = base_prompt[:60] + ("…" if len(base_prompt) > 60 else "")
        print(f"Prompt #{i}: {short_prompt}")
        print(f"   [{mod_name}] {summary_only}\n")

Prompt #1: What are the health benefits of green tea?
   [summary]  What are the health benefits of green tea? Green tea is one of the healthiest tea in the world .

Prompt #2: Explain the process of photosynthesis.
   [summary]  Explain the process of photosynthesis . Tell us about photosynthesis and how it takes place in plants .

Prompt #3: How can I improve my time management skills?
   [summary]  How can I improve my time management skills? How do you improve your time management?



In [4]:
title = "Печатаем summary для всех промптов"

print("\n" + "=" * 80)                       # верхняя рамка
print(title.center(80))                     # центрируем на 80 символов
print("=" * 80 + "\n")     


                       Печатаем summary для всех промптов                       



In [5]:
title = "Печатаем summary для всех промптов"

print("\n" + "=" * 80)                       # верхняя рамка
print(title.center(80))                     # центрируем на 80 символов
print("=" * 80 + "\n")     

# ----------  Выводим summary для каждого промпта  ----------
for i, base_prompt in enumerate(PROMPTS, 1):
    # ищем именно модификацию 'summary'
    for mod_name, mod_fn in MODS:
        if mod_name != "summary":
            continue                # пропускаем остальные модификации

        # 1. получаем строку с префиксом
        summary_with_prefix = mod_fn(base_prompt)

        # 2. убираем CUSTOM_WORDS
        summary_only = summary_with_prefix[len(CUSTOM_WORDS):]

        # 3. печатаем
        #short_prompt = base_prompt[:60] + ("…" if len(base_prompt) > 60 else "")
        print(f"🔹 PROMPT #{i}")
        print(f"📝 {base_prompt}")
        #print(f"Prompt #{i}: {short_prompt}")
        print(f"   [{mod_name}] {summary_only}\n")


                       Печатаем summary для всех промптов                       

🔹 PROMPT #1
📝 What are the health benefits of green tea?
   [summary]  What are the health benefits of green tea? Green tea is one of the healthiest tea in the world .

🔹 PROMPT #2
📝 Explain the process of photosynthesis.
   [summary]  Explain the process of photosynthesis . Tell us about photosynthesis and how it takes place in plants .

🔹 PROMPT #3
📝 How can I improve my time management skills?
   [summary]  How can I improve my time management skills? How do you improve your time management?



In [6]:
title = "Печатаем для каждого промпта сам промпт и все его модификации"

print("\n" + "=" * 80)                       # верхняя рамка
print(title.center(80))                     # центрируем на 80 символов
print("=" * 80 + "\n")     
# печатаем все ориганилы промптов и для них все их модификации, промпт за промптом
for i, base_prompt in enumerate(PROMPTS, 1):
    # 🔹 Печатаем исходный текст промпта (можно обрезать при необходимости)
    print(f"🔹 PROMPT #{i}")
    print(f"📝 {base_prompt}")

    # 🔹 Перебираем *все* модификации, кроме 'original'
    for mod_name, mod_fn in MODS:
        if mod_name == "original":
            continue  # исходник уже выше

        mod_text = mod_fn(base_prompt)

        # Убираем служебный префикс, если это summary
        if mod_name == "summary":
            mod_text = mod_text[len(CUSTOM_WORDS):]

        # Красивый вывод модификации
        print(f"   ➡️ [{mod_name}] {mod_text}")


         Печатаем для каждого промпта сам промпт и все его модификации          

🔹 PROMPT #1
📝 What are the health benefits of green tea?
   ➡️ [typo first e] What ar3 the health benefits of green tea?
   ➡️ [add salutation] Dear user, What are the health benefits of green tea?
   ➡️ [префикс ======] ==========What are the health benefits of green tea?
   ➡️ [префикс вопрос] I have a question. What are the health benefits of green tea?
   ➡️ [суффикс 10 лет?] What are the health benefits of green tea? in the next decade?
   ➡️ [summary]  What are the health benefits of green tea? Green tea is one of the healthiest tea in the world .
🔹 PROMPT #2
📝 Explain the process of photosynthesis.
   ➡️ [typo first e] Explain th3 process of photosynthesis.
   ➡️ [add salutation] Dear user, Explain the process of photosynthesis.
   ➡️ [префикс ======] ==========Explain the process of photosynthesis.
   ➡️ [префикс вопрос] I have a question. Explain the process of photosynthesis.
   ➡️ [суффикс 10 

In [7]:
topk_rows = []

for prompt in PROMPTS:                         # ← внешний цикл по всем промптам
    for mod_name, mod_fn in MODS:              #   внутренний — по модификациям
        mod_prompt = mod_fn(prompt)            #   теперь текущий prompt, а не [0]
        dists = _multi_step(mod_prompt, STEPS)

        for step_idx, probs in enumerate(dists, start=1):
            topk = torch.topk(probs, 5)
            for rank, tok_id in enumerate(topk.indices.tolist(), start=1):
                topk_rows.append({
                    "Prompt": prompt[:60] + ("…" if len(prompt) > 60 else ""),
                    "Step":   step_idx,
                    "Mod":    mod_name,
                    "Rank":   rank,
                    "Token":  _tokenizer.decode([tok_id]),
                    "Prob":   topk.values.tolist()[rank-1],
                })

topk_df = pd.DataFrame(topk_rows)


In [8]:
STEPS

3

In [9]:
MODS

[('original', <function __main__.<lambda>(p)>),
 ('typo first e', <function __main__.<lambda>(p)>),
 ('add salutation', <function __main__.<lambda>(p)>),
 ('префикс ======', <function __main__.<lambda>(p)>),
 ('префикс вопрос', <function __main__.<lambda>(p)>),
 ('суффикс 10 лет?', <function __main__.<lambda>(p)>),
 ('summary', <function __main__.<lambda>(p)>)]

In [10]:

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

if USE_FILE_PROMPTS == False:
    TRUNC = 60
else:
        TRUNC = 100   
TOP_K  = 5       # сколько токенов берём на шаг
records = []

# ---------- A. Сбор топ‑K для каждого prompt × mod × step ----------
for prompt in PROMPTS:
    for mod_name, mod_fn in MODS:
        mod_prompt = mod_fn(prompt)
        dists = _multi_step(mod_prompt, STEPS)          # твоя функция

        for step_idx, probs in enumerate(dists, start=1):
            topk = torch.topk(probs, TOP_K)             # top‑5 вероятных токенов
            for rank, (tok_id, prob) in enumerate(zip(topk.indices, topk.values), start=1):
                records.append({
                    "PromptFull": prompt,
                    "Prompt":     prompt[:TRUNC] + ("…" if len(prompt) > TRUNC else ""),
                    "Step":       step_idx,
                    "Rank":       rank,                 # 1 … 5
                    "Mod":        mod_name,
                    "Token":      _tokenizer.decode([tok_id]),
                    "Prob":       prob.item()
                })

topk_df = pd.DataFrame(records)

# ---------- B. Делаем длинный формат: Metric = {Token, Prob} ----------
long_df = topk_df.melt(
    id_vars    = ["Prompt", "Step", "Rank", "Mod"],
    value_vars = ["Token", "Prob"],
    var_name   = "Metric",
    value_name = "Val"
)

# --- C. Pivot -------------------------------------------
pivot_df = long_df.pivot_table(
    index   = ["Prompt", "Step", "Rank"],
    columns = ["Mod", "Metric"],
    values  = "Val",
    aggfunc = "first",
    sort    = False        # ← сохраняем порядок Token, Prob
)

# 1️⃣ Сортируем ТОЛЬКО строки → (Prompt → Step → Rank)
pivot_df = pivot_df.sort_index()

# 2️⃣ Доп. гарантия: явная перестановка колонок внутри MultiIndex
from itertools import chain
mods = [mod for mod in pivot_df.columns.levels[0]]          # все модификации в порядке level‑0
ordered_pairs = list(chain.from_iterable(
    ((m, "Token"), (m, "Prob")) for m in mods
))

pivot_df = pivot_df[ordered_pairs]  # ← теперь точно Token, Prob, Token, Prob …

# ---------- D. Вывод по каждому промпту отдельно ----------
for prompt_text, grp in pivot_df.groupby(level=0):      # группируем по Prompt
    print("🔹 PROMPT:\n", prompt_text, "\n")
    display(grp.droplevel(0))                           # оставляем Step, Rank
    print("-"*100 + "\n")


🔹 PROMPT:
 Explain the process of photosynthesis. 



Mod       original           typo first e           add salutation            \
Metric       Token      Prob        Token      Prob          Token      Prob   
Step Rank                                                                      
1    1          \n  0.234052           \n  0.246266             \n  0.345561   
     2         The  0.067022          The  0.056622           \n\n  0.035456   
     3        This  0.032585         This  0.038519           This   0.02831   
     4          It  0.027806           It  0.024726              I  0.026636   
     5        \n\n  0.017775            (  0.022258            The  0.022382   
2    1          \n  0.996881           \n  0.993396             \n  0.990645   
     2         The  0.000293          The  0.000639            The  0.000646   
     3           A  0.000133            A   0.00022              I  0.000571   
     4           I  0.000078         This  0.000198              A  0.000334   
     5          In  0.000075            I   0.00016           This  0.000322   
3    1         The  0.051376          The  0.052386            The  0.044798   
     2          In  0.018007         This  0.019969           This  0.029198   
     3        Step  0.017483           In   0.01718            How  0.027585   
     4         How  0.016146            A  0.014087           Step   0.02659   
     5        This  0.014691         Step  0.012585              I  0.023697   

Mod       префикс ======           префикс вопрос           суффикс 10 лет?  \
Metric             Token      Prob          Token      Prob           Token   
Step Rank                                                                     
1    1                \n  0.200959             \n  0.185765              \n   
     2               The  0.057674           What  0.066285             The   
     3              \n\n  0.051048            How  0.064829               I   
     4              This  0.023695              I  0.048763            \n\n   
     5                It  0.023055             It  0.041298              It   
2    1                \n   0.98451             \n  0.996097              \n   
     2               The  0.001404              A  0.000324             The   
     3                 A  0.000595              I  0.000269               A   
     4                In  0.000342            The  0.000256               I   
     5              This  0.000337           This  0.000089            This   
3    1               The  0.053258            The  0.034088             The   
     2              Step  0.053248              I  0.032326              In   
     3                 1   0.02312              A  0.030136               I   
     4                 A  0.020393           What  0.022832               A   
     5               How  0.018554            How  0.016062            This   

Mod                 summary            
Metric         Prob   Token      Prob  
Step Rank                              
1    1      0.20725      \n  0.177593  
     2     0.025421    Tell  0.070943  
     3     0.015973     How  0.023405  
     4     0.014594       .  0.020506  
     5     0.013529     The  0.019556  
2    1     0.995004      \n  0.920332  
     2     0.000462       A  0.003922  
     3      0.00027       �  0.002584  
     4      0.00023     The  0.002074  
     5     0.000184       R  0.001936  
3    1     0.059114       :  0.045004  
     2      0.02633       �  0.032531  
     3     0.025258       M  0.029343  
     4      0.02354       K  0.029014  
     5     0.021859       R  0.021639

----------------------------------------------------------------------------------------------------

🔹 PROMPT:
 How can I improve my time management skills? 



Mod       original           typo first e           add salutation            \
Metric       Token      Prob        Token      Prob          Token      Prob   
Step Rank                                                                      
1    1          \n  0.694234           \n  0.606448             \n  0.479521   
     2        \n\n  0.019273            I  0.029973           \n\n  0.038997   
     3           I  0.019261         \n\n  0.023605              I  0.027457   
     4         How  0.010069          How  0.014081         Please  0.016529   
     5         You  0.008694          You  0.010005         Answer  0.014901   
2    1          \n  0.997193           \n   0.99464             \n   0.99765   
     2         The  0.000231            I  0.000468            The    0.0002   
     3           I  0.000154          The  0.000429              A  0.000176   
     4           A  0.000145            A  0.000292              I  0.000171   
     5          In  0.000085        There  0.000183           This  0.000073   
3    1         You  0.070801          You  0.056385            The    0.0484   
     2          If  0.042039            I  0.046175            You  0.047107   
     3         The  0.041553           If   0.03875             If  0.037636   
     4       There  0.030015          The  0.037487              I  0.031937   
     5           I  0.027547        There  0.028399           This  0.028505   

Mod       префикс ======           префикс вопрос           суффикс 10 лет?  \
Metric             Token      Prob          Token      Prob           Token   
Step Rank                                                                     
1    1                \n  0.098707             \n   0.29335              \n   
     2                **  0.034066              I  0.112879             How   
     3              \n\n  0.025679            How  0.039823               I   
     4                 [  0.022648           What  0.024097            What   
     5                 I  0.018209             If  0.022703              in   
2    1                \n  0.997114             \n  0.995732              \n   
     2               The  0.000234              I  0.000448               I   
     3                 A  0.000208            The   0.00031             The   
     4                 I  0.000095              A  0.000232               A   
     5             There   0.00008             In  0.000094              In   
3    1               You  0.055118              I  0.090376             The   
     2               The  0.033346            The  0.042202              In   
     3                If  0.031071         Answer  0.042011               I   
     4                 1  0.024107             If  0.025898              If   
     5             There  0.022554            You  0.024743             You   

Mod                 summary            
Metric         Prob   Token      Prob  
Step Rank                              
1    1     0.226989      \n  0.304704  
     2     0.041623       I  0.039468  
     3      0.04034     How  0.033269  
     4     0.030512       [  0.022451  
     5     0.016892    What  0.020659  
2    1     0.991862      \n  0.996883  
     2     0.000547       A  0.000152  
     3     0.000533     The  0.000143  
     4     0.000285       I  0.000139  
     5     0.000243       T  0.000074  
3    1     0.043119       �  0.070892  
     2     0.040023       K  0.026471  
     3      0.03567       M  0.025139  
     4     0.029874       A  0.018681  
     5     0.028323       I  0.017636

----------------------------------------------------------------------------------------------------

🔹 PROMPT:
 What are the health benefits of green tea? 



Mod       original           typo first e           add salutation            \
Metric       Token      Prob        Token      Prob          Token      Prob   
Step Rank                                                                      
1    1          \n  0.562007           \n  0.382165             \n  0.342808   
     2         The   0.02335            I  0.022884              I  0.032325   
     3          It  0.014384          The   0.02164           \n\n  0.021421   
     4        Well  0.013971         Well  0.020851           What  0.018998   
     5        What  0.013664           It  0.015558              (  0.018357   
2    1          \n  0.995702           \n  0.957113             \n  0.993636   
     2         The  0.000393          The   0.00235              I  0.000589   
     3           A  0.000352            I  0.002172              A  0.000518   
     4           I  0.000162            A  0.001447            The  0.000425   
     5          In  0.000088         Well  0.001057           This  0.000152   
3    1       Green  0.264314        Green  0.096087          Green  0.107879   
     2         The  0.063577          The  0.064317              I  0.075249   
     3           A  0.026279            I  0.035921            The  0.066554   
     4          It  0.023792            A  0.032228              A  0.031053   
     5       There  0.020968           It  0.025878             It   0.02292   

Mod       префикс ======           префикс вопрос           суффикс 10 лет?  \
Metric             Token      Prob          Token      Prob           Token   
Step Rank                                                                     
1    1                \n  0.189409             \n  0.232792              \n   
     2              \n\n  0.030432              I  0.102887            What   
     3               The  0.026373             It  0.039677             and   
     4               The  0.023442           What  0.033184             How   
     5              Well  0.019513             Is   0.02866             The   
2    1                \n  0.978224             \n  0.995518              \n   
     2                 A  0.002657              A    0.0004             The   
     3               The  0.001728              I  0.000364               I   
     4             Green  0.000737            The  0.000329               A   
     5             There   0.00063             It  0.000107            This   
3    1             Green  0.309205              I  0.089385           Green   
     2               The   0.06762            The  0.054453             The   
     3                 A   0.03092          Green  0.037853              In   
     4            Health  0.027654              A  0.034264               I   
     5             There  0.020356             It  0.031126               A   

Mod                    summary            
Metric         Prob      Token      Prob  
Step Rank                                 
1    1     0.256441         It  0.149229  
     2     0.037819         \n  0.071055  
     3     0.027339        The  0.068596  
     4     0.023479      Green  0.044514  
     5      0.01693      There  0.025969  
2    1     0.973675         is  0.324026  
     2     0.001592   contains  0.208412  
     3     0.001145        has  0.156196  
     4     0.000744         's  0.096305  
     5     0.000616        can  0.020282  
3    1     0.068059          a  0.116188  
     2     0.066103       rich  0.078524  
     3     0.024888       also   0.05643  
     4     0.023861        one  0.046386  
     5     0.021685        the   0.04076

----------------------------------------------------------------------------------------------------



In [11]:
if USE_FILE_PROMPTS == False: # количество знаков, которое выводится из каждого промпта, если промпты  списком
    TRUNC = 60
else:
        TRUNC = 100    # количество знаков, которое выводится из каждого промпта, если промпт - внешний файл

records = []                                  # сюда складываем все строки будущего DF

for i, prompt in enumerate(PROMPTS):          # перебираем промпты
    base_dists = _multi_step(prompt, STEPS)   # «базовый» вариант без модификации
    for mod_name, mod_fn in MODS:             # перебираем модификации
        if mod_name == "original":            # «original» сравнивать не с чем
            continue

        mod_prompt = mod_fn(prompt)
        mod_dists = _multi_step(mod_prompt, STEPS)

        for step in range(STEPS):             # перебираем шаги
            records.append({
                "PromptFull": prompt,    
                #"Prompt": f"prompt {i+1}",       # строковое имя промпта
                "Prompt":     prompt[:TRUNC] + ("…" if len(prompt) > TRUNC else ""),
                
                "Step":   step + 1,              # шаг начинается с 1
                "Mod":    mod_name,              # название модификации
                "KL":     _kl (base_dists[step], mod_dists[step]),
                "Cos":    _cos(base_dists[step], mod_dists[step]),
            })


In [12]:
df = pd.DataFrame(records)

In [13]:
# (1) глобальная настройка вывода -----------------
pd.set_option("display.max_columns", None)   # показывать все колонки
# ----- 1. Cosine ------------------------------------------------
cos_df = df.pivot_table(
    index   = ["Prompt", "Step"],     # мульти‑индекс: промпт → шаг
    columns = "Mod",                  # колонки — модификации
    values  = "Cos"                   # берём только Cos
).sort_index()

# ----- 2. KL ----------------------------------------------------
kl_df = df.pivot_table(
    index   = ["Prompt", "Step"],
    columns = "Mod",
    values  = "KL"
).sort_index()

# ----- 3. Печать ------------------------------------------------
from IPython.display import display

print("◾️ Cosine similarity")
display(cos_df)

print("\n◾️ KL divergence")
display(kl_df)

◾️ Cosine similarity


Mod                                                add salutation   summary  \
Prompt                                       Step                             
Explain the process of photosynthesis.       1           0.967754  0.891455   
                                             2           1.000000  0.999970   
                                             3           0.876599  0.280181   
How can I improve my time management skills? 1           0.996153  0.979866   
                                             2           1.000000  1.000000   
                                             3           0.916155  0.196492   
What are the health benefits of green tea?   1           0.992205  0.411159   
                                             2           1.000000  0.000047   
                                             3           0.824285  0.000003   

Mod                                                typo first e  \
Prompt                                       Step                 
Explain the process of photosynthesis.       1         0.994649   
                                             2         1.000000   
                                             3         0.966150   
How can I improve my time management skills? 1         0.999550   
                                             2         1.000000   
                                             3         0.964912   
What are the health benefits of green tea?   1         0.997001   
                                             2         0.999991   
                                             3         0.865486   

Mod                                                префикс ======  \
Prompt                                       Step                   
Explain the process of photosynthesis.       1           0.980456   
                                             2           0.999999   
                                             3           0.883189   
How can I improve my time management skills? 1           0.803384   
                                             2           1.000000   
                                             3           0.919023   
What are the health benefits of green tea?   1           0.949445   
                                             2           0.999995   
                                             3           0.997217   

Mod                                                префикс вопрос  \
Prompt                                       Step                   
Explain the process of photosynthesis.       1           0.865905   
                                             2           1.000000   
                                             3           0.741423   
How can I improve my time management skills? 1           0.923956   
                                             2           1.000000   
                                             3           0.697959   
What are the health benefits of green tea?   1           0.891002   
                                             2           1.000000   
                                             3           0.495334   

Mod                                                суффикс 10 лет?  
Prompt                                       Step                   
Explain the process of photosynthesis.       1            0.971476  
                                             2            1.000000  
                                             3            0.874893  
How can I improve my time management skills? 1            0.951940  
                                             2            1.000000  
                                             3            0.868246  
What are the health benefits of green tea?   1            0.976160  
                                             2            0.999997  
                                             3            0.796396


◾️ KL divergence


Mod                                                add salutation    summary  \
Prompt                                       Step                              
Explain the process of photosynthesis.       1           0.314789   0.588835   
                                             2           0.003581   0.071641   
                                             3           0.284424   1.626293   
How can I improve my time management skills? 1           0.177653   0.505696   
                                             2           0.000233   0.001128   
                                             3           0.217192   1.796814   
What are the health benefits of green tea?   1           0.244052   1.167441   
                                             2           0.001152  10.810578   
                                             3           0.216445  12.755248   

Mod                                                typo first e  \
Prompt                                       Step                 
Explain the process of photosynthesis.       1         0.105580   
                                             2         0.001366   
                                             3         0.103833   
How can I improve my time management skills? 1         0.045023   
                                             2         0.001017   
                                             3         0.119258   
What are the health benefits of green tea?   1         0.174233   
                                             2         0.030690   
                                             3         0.242865   

Mod                                                префикс ======  \
Prompt                                       Step                   
Explain the process of photosynthesis.       1           0.212915   
                                             2           0.008343   
                                             3           0.236056   
How can I improve my time management skills? 1           1.381405   
                                             2           0.000309   
                                             3           0.210952   
What are the health benefits of green tea?   1           0.750038   
                                             2           0.011952   
                                             3           0.053523   

Mod                                                префикс вопрос  \
Prompt                                       Step                   
Explain the process of photosynthesis.       1           0.579362   
                                             2           0.000782   
                                             3           0.735712   
How can I improve my time management skills? 1           0.472592   
                                             2           0.000703   
                                             3           0.342549   
What are the health benefits of green tea?   1           0.390653   
                                             2           0.000540   
                                             3           0.542304   

Mod                                                суффикс 10 лет?  
Prompt                                       Step                   
Explain the process of photosynthesis.       1            0.514599  
                                             2            0.001002  
                                             3            0.385161  
How can I improve my time management skills? 1            0.697250  
                                             2            0.002839  
                                             3            0.137906  
What are the health benefits of green tea?   1            0.504358  
                                             2            0.015690  
                                             3            0.288145

In [14]:
# --- TOP‑5 по шагам в компактном виде (то есть представление ячейки 7 в более компактном виде)
# не очень понятно, нужно ли, выше то же самое более детально
topk_dfs = []

for prompt, g in topk_df.groupby("Prompt"):
    # g: строки для одного промпта
    # соберём список 5 токенов строкой "tok1, tok2, …"
    g_sorted = g.sort_values(["Step", "Mod", "Rank"])
    g_sorted["TokList"] = g_sorted.groupby(["Step", "Mod"])["Token"]\
                                  .transform(lambda s: ", ".join(s))
    tidy = (g_sorted.drop_duplicates(subset=["Step", "Mod"])
                    .pivot(index="Step", columns="Mod", values="TokList"))
    tidy.index = [f"Step {i}" for i in tidy.index]
    topk_dfs.append((prompt, tidy))

# Печатаем

show_prompt=100 #показываем, сколько символов из промпта мы выводим перед таблицей

for i, (prompt, tidy) in enumerate(topk_dfs, 1):
    prev = prompt.replace("\n", " ")[:show_prompt] + ("…" if len(prompt) > show_prompt else "")
    print(f"\n\n📌 PROMPT {i}: {prev}")
    display(tidy)




📌 PROMPT 1: Explain the process of photosynthesis.


Mod,add salutation,original,summary,typo first e,префикс ======,префикс вопрос,суффикс 10 лет?
Step 1,"\n, \n\n, This, I, The","\n, The, This, It, \n\n","\n, Tell, How, ., The","\n, The, This, It, (","\n, The, \n\n, This, It","\n, What, How, I, It","\n, The, I, \n\n, It"
Step 2,"\n, The, I, A, This","\n, The, A, I, In","\n, A, �, The, R","\n, The, A, This, I","\n, The, A, In, This","\n, A, I, The, This","\n, The, A, I, This"
Step 3,"The, This, How, Step, I","The, In, Step, How, This",":, �, M, K, R","The, This, In, A, Step","The, Step, 1, A, How","The, I, A, What, How","The, In, I, A, This"




📌 PROMPT 2: How can I improve my time management skills?


Mod,add salutation,original,summary,typo first e,префикс ======,префикс вопрос,суффикс 10 лет?
Step 1,"\n, \n\n, I, Please, Answer","\n, \n\n, I, How, You","\n, I, How, [, What","\n, I, \n\n, How, You","\n, **, \n\n, [, I","\n, I, How, What, If","\n, How, I, What, in"
Step 2,"\n, The, A, I, This","\n, The, I, A, In","\n, A, The, I, T","\n, I, The, A, There","\n, The, A, I, There","\n, I, The, A, In","\n, I, The, A, In"
Step 3,"The, You, If, I, This","You, If, The, There, I","�, K, M, A, I","You, I, If, The, There","You, The, If, 1, There","I, The, Answer, If, You","The, In, I, If, You"




📌 PROMPT 3: What are the health benefits of green tea?


Mod,add salutation,original,summary,typo first e,префикс ======,префикс вопрос,суффикс 10 лет?
Step 1,"\n, I, \n\n, What, (","\n, The, It, Well, What","It, \n, The, Green, There","\n, I, The, Well, It","\n, \n\n, The, The, Well","\n, I, It, What, Is","\n, What, and, How, The"
Step 2,"\n, I, A, The, This","\n, The, A, I, In","is, contains, has, 's, can","\n, The, I, A, Well","\n, A, The, Green, There","\n, A, I, The, It","\n, The, I, A, This"
Step 3,"Green, I, The, A, It","Green, The, A, It, There","a, rich, also, one, the","Green, The, I, A, It","Green, The, A, Health, There","I, The, Green, A, It","Green, The, In, I, A"
